# Usage

The basic usage of cmomy is to manipulate central moments.  
We measure two quantities pretend quantities.  The 'energy' and the 'position' of a thing.  We'll construct the average value for each record. Lets say 100 samples each.  

In [1]:
import numpy as np

np.set_printoptions(precision=4)

In [2]:
import numpy as np
import xarray as xr
import cmomy
from IPython.display import display


np.random.seed(0)

nsamp = 100
energy = xr.DataArray(np.random.rand(nsamp), dims=["samp"])

position = xr.DataArray(np.random.rand(nsamp, 3), dims=["samp", "dim"])

# weight assicated with each sample and each record
w = xr.DataArray(np.random.rand(nsamp), dims=["samp"])

# average over the samples
ce = cmomy.xCentralMoments.from_vals(x=energy, w=w, dim="samp", mom=3)
cp = cmomy.xCentralMoments.from_vals(x=position, w=w, dim="samp", mom=3)

print("energy")
display(ce)

print("position")
display(cp)

energy


<xCentralMoments(val_shape=(), mom=(3,))>
<xarray.DataArray (mom_0: 4)>
array([4.9972e+01, 4.7186e-01, 7.9673e-02, 2.7133e-03])
Dimensions without coordinates: mom_0

position


<xCentralMoments(val_shape=(3,), mom=(3,))>
<xarray.DataArray (dim: 3, mom_0: 4)>
array([[ 4.9972e+01,  5.3881e-01,  8.7283e-02, -7.5029e-03],
       [ 4.9972e+01,  5.0714e-01,  7.7545e-02,  2.6483e-03],
       [ 4.9972e+01,  5.0251e-01,  9.4424e-02, -1.4499e-03]])
Dimensions without coordinates: dim, mom_0

Here, we've used the constructor {meth}`cmomy.CentralMoments.from_vals`.  There are a host of other constructors to create {class}`cmomy.CentralMoments` object.  Take a look at the docs for further info.

## Basic attributes

Notice that there are three `shape` parameters associated with a {class}`cmomy.CentralMoments` object:
 
* {attr}`cmomy.CentralMoments.mom_shape` : shape of the moments.  For single variable, tuple (mom+1,).  For comoments, (mom_0+1, mom_1+1)
* {attr}`cmomy.CentralMoments.val_shape`: shape of the 'values' part of the data.  For scalar data, `val_shape = ()`.  For vector data, this is the shape of the passed observation data.
* {attr}`cmomy.CentralMoments.shape`: total shape of wrapped moments `shape = val_shape + mom_shape`

In [3]:
for name, c in {"energy": ce, "position": cp}.items():
    print(
        f"""
{name} shapes:
    mom_shape: {c.mom_shape}
    val_shape: {c.val_shape}
    shape    : {c.shape}
"""
    )


energy shapes:
    mom_shape: (4,)
    val_shape: ()
    shape    : (4,)


position shapes:
    mom_shape: (4,)
    val_shape: (3,)
    shape    : (3, 4)



To access the underlying data, use the {attr}`cmomy.CentralMoments.values` attribute.  The structure of `values` is:

* `values[...,0]`: total weight
* `values[...,1]`: average/mean value
* `values[...,k>1]`: `kth` central moment.

In [4]:
ce.values

<xarray.DataArray (mom_0: 4)>
array([4.9972e+01, 4.7186e-01, 7.9673e-02, 2.7133e-03])
Dimensions without coordinates: mom_0

To access all the central moments (zeroth and first included), use the {meth}`cmomy.CentralMoments.cmom` method.

In [5]:
ce.cmom()

<xarray.DataArray (mom_0: 4)>
array([1.    , 0.    , 0.0797, 0.0027])
Dimensions without coordinates: mom_0

Likewise, the central moments can be converted to raw moments using the {meth}`cmomy.CentralMoments.rmom` method.  This uses the {mod}`cmomy.convert` module behind the scenes.  

In [6]:
# <x**k>
ce.rmom()

<xarray.DataArray (mom_0: 4)>
array([1.    , 0.4719, 0.3023, 0.2206])
Dimensions without coordinates: mom_0

Additionally, there are {class}`xarray.DataArray` like attributes

In [7]:
ce.coords

Coordinates:
    *empty*

In [8]:
ce.attrs

{}

In [9]:
ce.sizes

Frozen({'mom_0': 4})

In [10]:
ce.dims

('mom_0',)

## Manipulating (co)moments

So we have our averages.  Cool.  Not very special.  But what if instead we repeat our experiment.  Let's say we did the experiment 10 times each time giving a single record.   Then our data would look like

In [11]:
nsamp = 100
nrec = 10
energy = xr.DataArray(np.random.rand(nrec, nsamp), dims=["rec", "samp"])
position = xr.DataArray(np.random.rand(nrec, nsamp, 3), dims=["rec", "samp", "dim"])

# weight assicated with each sample and each record
w = xr.DataArray(np.random.rand(nrec, nsamp), dims=["rec", "samp"])

# average over the samples
ce = cmomy.xCentralMoments.from_vals(x=energy, w=w, dim="samp", mom=3)
cp = cmomy.xCentralMoments.from_vals(x=position, w=w, dim="samp", mom=3)

Consider just the energy.  We suspect that there is some correlation between the experiments (they where done in rapid succession).  So we'd like to consider two adjacent experiments as one experiment.  For this, we can use the {meth}`cmomy.xCentralMoments.block` method.

In [12]:
ce

<xCentralMoments(val_shape=(10,), mom=(3,))>
<xarray.DataArray (rec: 10, mom_0: 4)>
array([[ 4.9492e+01,  5.0085e-01,  9.5395e-02,  6.3896e-05],
       [ 4.7085e+01,  4.3387e-01,  8.1477e-02,  8.5081e-03],
       [ 4.5617e+01,  4.5139e-01,  9.1416e-02,  1.0999e-02],
       [ 5.1835e+01,  5.5002e-01,  6.9482e-02, -3.0315e-03],
       [ 4.7215e+01,  5.3124e-01,  8.7490e-02, -3.5280e-03],
       [ 4.7843e+01,  4.7763e-01,  9.5671e-02, -2.0765e-05],
       [ 4.4453e+01,  4.8569e-01,  8.4392e-02,  6.7738e-03],
       [ 5.4015e+01,  5.0624e-01,  9.4580e-02, -9.7587e-04],
       [ 4.8715e+01,  5.4611e-01,  8.5452e-02, -7.4777e-03],
       [ 5.2069e+01,  4.9491e-01,  9.1639e-02,  1.7960e-03]])
Dimensions without coordinates: rec, mom_0

In [13]:
ce.block(block_size=2, dim="rec")

<xCentralMoments(val_shape=(5,), mom=(3,))>
<xarray.DataArray (rec: 5, mom_0: 4)>
array([[ 9.6577e+01,  4.6819e-01,  8.9730e-02,  4.8777e-03],
       [ 9.7451e+01,  5.0385e-01,  8.2171e-02,  1.9050e-03],
       [ 9.5058e+01,  5.0426e-01,  9.2326e-02, -2.0914e-03],
       [ 9.8468e+01,  4.9696e-01,  9.0085e-02,  2.6780e-03],
       [ 1.0078e+02,  5.1966e-01,  8.9303e-02, -2.9227e-03]])
Dimensions without coordinates: rec, mom_0

Instead, we can resample the already averaged data using {meth}`cmomy.xCentralMoments.resample_and_reduce`.  We produce a 20 new samples taken from the original (averaged) data.

In [14]:
ce_resamp = ce.resample_and_reduce(nrep=20, dim="rec")
ce_resamp

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


<xCentralMoments(val_shape=(20,), mom=(3,))>
<xarray.DataArray (rep: 20, mom_0: 4)>
array([[ 4.8428e+02,  4.8966e-01,  8.8854e-02,  1.5749e-03],
       [ 4.8757e+02,  4.9211e-01,  8.9921e-02,  1.9170e-03],
       [ 4.7916e+02,  4.9800e-01,  8.8011e-02,  1.1529e-03],
       [ 4.9549e+02,  4.9436e-01,  8.9036e-02,  1.0334e-03],
       [ 4.9721e+02,  4.9257e-01,  8.9668e-02,  1.1649e-03],
       [ 4.8938e+02,  4.9266e-01,  9.2495e-02,  8.7200e-04],
       [ 4.8938e+02,  5.1619e-01,  8.3040e-02, -2.0161e-03],
       [ 5.0642e+02,  5.1115e-01,  8.4668e-02, -2.9762e-04],
       [ 4.9055e+02,  4.9460e-01,  8.6275e-02,  1.4601e-03],
       [ 4.8885e+02,  4.9954e-01,  9.0073e-02,  3.2957e-04],
       [ 5.1368e+02,  5.0835e-01,  8.3207e-02, -2.5765e-04],
       [ 4.7901e+02,  4.9636e-01,  8.9123e-02,  1.8429e-03],
       [ 4.8153e+02,  4.8299e-01,  9.1768e-02,  2.6249e-03],
       [ 5.0477e+02,  5.0003e-01,  8.6921e-02,  7.3117e-04],
       [ 4.7761e+02,  4.9914e-01,  8.7068e-02,  2.5504e-03],
       [ 4.9999e+02,  4.9722e-01,  9.1266e-02,  9.0316e-04],
       [ 4.6737e+02,  4.8319e-01,  9.0054e-02,  4.7384e-03],
       [ 4.7983e+02,  4.7829e-01,  9.2894e-02,  4.9755e-03],
       [ 4.8614e+02,  5.0413e-01,  9.0713e-02,  7.3786e-04],
       [ 4.9457e+02,  5.2369e-01,  8.6780e-02, -2.6871e-03]])
Dimensions without coordinates: rep, mom_0

This is different than the usual 'resample values'.  This is also available if the original data is available.

In [15]:
a = energy.stack(c=["rec", "samp"])

In [16]:
b = w.stack(c=["rec", "samp"])

In [17]:
# consider 'all' the data for this
out = cmomy.xCentralMoments.from_resample_vals(
    energy.stack(c=["rec", "samp"]),
    w=w.stack(c=["rec", "samp"]),
    dim="c",
    nrep=20,
    mom=3,
)
out

<xCentralMoments(val_shape=(20,), mom=(3,))>
<xarray.DataArray (rep: 20, mom_0: 4)>
array([[ 4.7358e+02,  4.9846e-01,  9.1738e-02,  8.0991e-04],
       [ 4.8822e+02,  4.8258e-01,  9.0637e-02,  3.0800e-03],
       [ 4.9564e+02,  5.1132e-01,  9.2224e-02, -4.4936e-04],
       [ 4.8164e+02,  5.0897e-01,  8.4654e-02,  2.1622e-04],
       [ 4.9652e+02,  4.9354e-01,  9.3469e-02,  1.7070e-03],
       [ 4.7592e+02,  4.8516e-01,  9.0992e-02,  2.3010e-03],
       [ 4.8585e+02,  5.0387e-01,  8.8592e-02,  2.8266e-04],
       [ 5.0004e+02,  4.9166e-01,  8.8463e-02,  1.2245e-03],
       [ 4.7695e+02,  4.8966e-01,  9.1360e-02,  9.0917e-04],
       [ 5.0263e+02,  5.0681e-01,  8.3679e-02,  3.7414e-04],
       [ 4.7908e+02,  5.0516e-01,  8.3234e-02,  2.3921e-04],
       [ 4.6365e+02,  5.0937e-01,  8.8850e-02,  1.2741e-05],
       [ 4.8244e+02,  4.7714e-01,  8.7727e-02,  3.6657e-03],
       [ 4.9218e+02,  5.1718e-01,  8.8596e-02, -1.6255e-03],
       [ 4.7064e+02,  5.1144e-01,  9.0594e-02, -5.4300e-05],
       [ 5.0153e+02,  5.1652e-01,  9.1733e-02, -5.0623e-04],
       [ 4.8411e+02,  5.0143e-01,  8.7960e-02,  1.5813e-03],
       [ 4.8828e+02,  4.9680e-01,  8.9379e-02,  1.1217e-03],
       [ 4.8712e+02,  5.0964e-01,  8.6629e-02, -5.5439e-06],
       [ 4.6959e+02,  4.9808e-01,  8.6972e-02,  1.7025e-04]])
Coordinates:
    c        object (0, 0)
    rec      int64 0
    samp     int64 0
Dimensions without coordinates: rep, mom_0

We see that the deviation in the moments is similar using the two resampling methods:

In [18]:
out.values.sel(mom_0=slice(1, None)).std("rep")

<xarray.DataArray (mom_0: 3)>
array([0.0111, 0.0028, 0.0012])
Coordinates:
    c        object (0, 0)
    rec      int64 0
    samp     int64 0
Dimensions without coordinates: mom_0

In [19]:
ce_resamp.values.sel(mom_0=slice(1, None)).std("rep")

<xarray.DataArray (mom_0: 3)>
array([0.0108, 0.0028, 0.0018])
Dimensions without coordinates: mom_0

We can also reduce our original data across all the records using {meth}`cmomy.xCentralMoments.reduce`

In [20]:
ce.reduce(dim="rec")

<xCentralMoments(val_shape=(), mom=(3,))>
<xarray.DataArray (mom_0: 4)>
array([4.8834e+02, 4.9875e-01, 8.8997e-02, 8.5016e-04])
Dimensions without coordinates: mom_0